In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv("Crop_recommendation.csv")
df.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   N            2200 non-null   int64  
 1   P            2200 non-null   int64  
 2   K            2200 non-null   int64  
 3   temperature  2200 non-null   float64
 4   humidity     2200 non-null   float64
 5   ph           2200 non-null   float64
 6   rainfall     2200 non-null   float64
 7   label        2200 non-null   object 
dtypes: float64(4), int64(3), object(1)
memory usage: 137.6+ KB


In [3]:
df.isnull().sum()
print(df['label'].value_counts())
df.head()

label
rice           100
maize          100
chickpea       100
kidneybeans    100
pigeonpeas     100
mothbeans      100
mungbean       100
blackgram      100
lentil         100
pomegranate    100
banana         100
mango          100
grapes         100
watermelon     100
muskmelon      100
apple          100
orange         100
papaya         100
coconut        100
cotton         100
jute           100
coffee         100
Name: count, dtype: int64


,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [4]:
import plotly.express as px
import plotly.io as pio

fig = px.histogram(df, x='temperature', color='label',
                       title=f'Distribution of {df["temperature"]} by Crop',
                       nbins=30, barmode='overlay')
fig.show()

c:\Users\Chandru P\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])
c:\Users\Chandru P\AppData\Local\Programs\Python\Python311\Lib\site-packages\kaleido\_sync_server.py:11: UserWarning:




This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.




In [5]:
npk_avg = df.groupby('label')[['N', 'P', 'K']].mean().reset_index()
npk_avg

,label,N,P,K
0,apple,20.80,134.22,199.89
1,banana,100.23,82.01,50.05
2,blackgram,40.02,67.47,19.24
3,chickpea,40.09,67.79,79.92
4,coconut,21.98,16.93,30.59
5,coffee,101.20,28.74,29.94
6,cotton,117.77,46.24,19.56
7,grapes,23.18,132.53,200.11
8,jute,78.40,46.86,39.99
9,kidneybeans,20.75,67.54,20.05


In [6]:
fig = px.bar(npk_avg, x='label', y=['N', 'P', 'K'],
             title='Average NPK Values by Crop',
             labels={'value': 'Amount', 'variable': 'Nutrient'},
             barmode='group')
fig.show()

c:\Users\Chandru P\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [7]:
env_avg = df.groupby('label')[['temperature', 'humidity', 'ph', 'rainfall']].mean().reset_index()
env_avg

,label,temperature,humidity,ph,rainfall
0,apple,22.630942,92.333383,5.929663,112.654779
1,banana,27.376798,80.358123,5.983893,104.626980
2,blackgram,29.973340,65.118426,7.133952,67.884151
3,chickpea,18.872847,16.860439,7.336957,80.058977
4,coconut,27.409892,94.844272,5.976562,175.686646
5,coffee,25.540477,58.869846,6.790308,158.066295
6,cotton,23.988958,79.843474,6.912675,80.398043
7,grapes,23.849575,81.875228,6.025937,69.611829
8,jute,24.958376,79.639864,6.732778,174.792798
9,kidneybeans,20.115085,21.605357,5.749411,105.919778


In [8]:
fig = px.bar(env_avg, x='label', y=['temperature', 'humidity', 'ph', 'rainfall'],
             title='Average Environmental Factors by Crop',
             labels={'value': 'Value', 'variable': 'Factor'},
             barmode='group')
fig.show()

c:\Users\Chandru P\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [9]:
X = df[['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']]
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(f"Accuracy Score: {accuracy_score(y_test, y_pred)*100:.2f}%")
cv_scores = cross_val_score(model, X_train, y_train, cv=5)
print(f"Cross-Validation Accuracy: {cv_scores.mean()*100:.2f}% ± {cv_scores.std()*100:.2f}%")

Accuracy Score: 99.32%
Cross-Validation Accuracy: 99.32% ± 0.58%


In [10]:
feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importance:")
print(feature_importances)


Feature Importance:
       Feature  Importance
4     humidity    0.220780
6     rainfall    0.220463
2            K    0.179947
1            P    0.144161
0            N    0.107895
3  temperature    0.075547
5           ph    0.051207


In [11]:
fig = px.bar(feature_importances,
             x='Importance',
             y='Feature',
             orientation='h',
             title='Feature Importance in Crop Prediction',
             text='Importance',
             color='Feature')
fig.update_traces(texttemplate='%{text:.3f}', textposition='outside')
fig.update_layout(showlegend=False, height=500)
fig.show()

c:\Users\Chandru P\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [12]:
from sklearn.metrics import classification_report

report = classification_report(y_test, y_pred, output_dict=True)

# Extract accuracy for each crop
crop_accuracies = []
for crop in model.classes_:
    if crop in report:
        crop_accuracies.append({
            'Crop': crop,
            'Precision': report[crop]['precision'] * 100,
            'Recall': report[crop]['recall'] * 100,
            'F1-Score': report[crop]['f1-score'] * 100
        })

crop_acc_df = pd.DataFrame(crop_accuracies)

fig = px.bar(crop_acc_df, x='Crop', y=['Precision', 'Recall', 'F1-Score'],
             title='Per-Crop Performance Metrics',
             barmode='group')
fig.show()

c:\Users\Chandru P\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [13]:
def predict_crop():

    try:
        #user inputs
        N = float(input("Enter Nitrogen (N) content (0-140): "))
        P = float(input("Enter Phosphorus (P) content (5-145): "))
        K = float(input("Enter Potassium (K) content (5-205): "))
        temperature = float(input("Enter Temperature (°C) (8-44): "))
        humidity = float(input("Enter Humidity (%) (14-100): "))
        ph = float(input("Enter pH value (3.5-10): "))
        rainfall = float(input("Enter Rainfall (mm) (20-300): "))

        #input dataframe
        user_input = pd.DataFrame({
            'N': [N],
            'P': [P],
            'K': [K],
            'temperature': [temperature],
            'humidity': [humidity],
            'ph': [ph],
            'rainfall': [rainfall]
        })

        #prediction
        prediction = model.predict(user_input)[0]
        prediction_proba = model.predict_proba(user_input)[0]
        confidence = max(prediction_proba) * 100

        #Display results
        print("\n")
        print("                    PREDICTION RESULTS")
        print(f"\n Recommended Crop: {prediction.upper()}")
        print(f" Confidence: {confidence:.2f}%")

        #Show top 3 predictions
        print("\n")
        print("Top 3 Crop Recommendations:")

        #top 3 predictions with probabilities
        top_indices = prediction_proba.argsort()[-3:][::-1]
        for i, idx in enumerate(top_indices, 1):
            crop_name = model.classes_[idx]
            prob = prediction_proba[idx] * 100
            print(f"{i}. {crop_name:15} - {prob:.2f}%")

        print("=" * 60)

        return prediction

    except ValueError:
        print("\n❌ Error: Please enter valid numerical values!")
        return None
    except Exception as e:
        print(f"\n❌ Error occurred: {str(e)}")
        return None

predicted_crop = predict_crop()



                    PREDICTION RESULTS

 Recommended Crop: MUSKMELON
 Confidence: 38.00%


Top 3 Crop Recommendations:
1. muskmelon       - 38.00%
2. chickpea        - 15.00%
3. coffee          - 13.00%


In [14]:
import joblib

joblib.dump(model, "crop_model.pkl")

['crop_model.pkl']

In [15]:
import joblib
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Load model
model = joblib.load("crop_model.pkl")

# Test prediction (replace with your own sample)
X = np.array([[90, 42, 43, 20.8, 82.0, 6.5, 202.9]])  # N, P, K, Temp, Hum, pH, Rainfall
prediction = model.predict(X)

print("Predicted Crop:", prediction[0])


Predicted Crop: rice
